In [5]:
import pandas as pd
import numpy as np

In [6]:
data = pd.read_excel("tumor-data-without-healthy.xlsx")

In [7]:
labels = [value for column, value in data.iloc[0].items()]
data = data.set_axis(labels, axis=1)
data = data[data.columns[1:]].apply(
    lambda x: pd.to_numeric(x.replace('<', '').replace(',', '.'), errors='coerce')
)

In [8]:
data.drop(index=0, inplace=True)
data["Гормональная активность 0-нет               1-да"] = data[
    "Гормональная активность 0-нет               1-да"].astype(str)
data["Гормональная активность 0-нет               1-да"] = data[
"Гормональная активность 0-нет               1-да"].fillna("missing")

In [9]:
""" dropping some columns """

data.dropna(axis=1, how="all", inplace=True)
data.dropna(axis=0, thresh=int(data.shape[1] * 0.6), inplace=True)
data.dropna(axis=1, thresh=int(data.shape[0] * 0.6), inplace=True)
data.drop(["Пол:    0-жен, 1-муж"], inplace=True, axis=1)
X = data.drop(columns=["КАН", "АКР"])
y = np.argmax(data[["КАН", "АКР"]], axis=1)

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, random_state=42)

In [18]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer

import optuna

def objective(trial):
    param = {
        'C': trial.suggest_float('C', 1e-1, 10.0, log=False),
        'gamma': trial.suggest_categorical('gamma', ['scale', 'auto']),
        'kernel': trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid']),
        'coef0': trial.suggest_float('coef0', 1e-1, 10.0, log=True)
    }

    pipeline = make_pipeline(
        SimpleImputer(strategy='median'),
        SVC(**param, random_state=42)
    )
    pipeline.fit(X_train, y_train)
    score = pipeline.score(X_test, y_test)
    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2024-12-22 14:26:58,387] A new study created in memory with name: no-name-b1a9644b-94af-4e71-b026-3970b9e44e0e
[I 2024-12-22 14:26:58,397] Trial 0 finished with value: 0.8181818181818182 and parameters: {'C': 0.7353891962688719, 'gamma': 'auto', 'kernel': 'poly', 'coef0': 4.763858242762223}. Best is trial 0 with value: 0.8181818181818182.
[I 2024-12-22 14:26:58,408] Trial 1 finished with value: 0.36363636363636365 and parameters: {'C': 1.3355281062152273, 'gamma': 'auto', 'kernel': 'sigmoid', 'coef0': 1.1182661789237487}. Best is trial 0 with value: 0.8181818181818182.
[I 2024-12-22 14:26:58,418] Trial 2 finished with value: 0.36363636363636365 and parameters: {'C': 9.628116981994937, 'gamma': 'auto', 'kernel': 'sigmoid', 'coef0': 0.5086409863135667}. Best is trial 0 with value: 0.8181818181818182.
[I 2024-12-22 14:26:58,427] Trial 3 finished with value: 0.36363636363636365 and parameters: {'C': 3.401873673558067, 'gamma': 'auto', 'kernel': 'sigmoid', 'coef0': 0.8646573468334025}. B

In [19]:
pipeline = make_pipeline(
    SimpleImputer(strategy='median'),
    SVC(**study.best_params)
)

In [20]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for i, (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
    print(f"Fold {i}:")
    xtrain, xtest = X_train.iloc[train_index], X_train.iloc[test_index]
    ytrain, ytest = y_train[train_index], y_train[test_index]
    pipeline.fit(xtrain, ytrain)

    y_pred = pipeline.predict(xtest)

    print("Accuracy:", accuracy_score(ytest, y_pred))
    print("F1:", f1_score(ytest, y_pred))
    print("Precision:", precision_score(ytest, y_pred))
    print("Recall:", recall_score(ytest, y_pred))
    print()

Fold 0:
Accuracy: 0.8421052631578947
F1: 0.8
Precision: 0.8571428571428571
Recall: 0.75

Fold 1:
Accuracy: 0.7777777777777778
F1: 0.7142857142857143
Precision: 0.8333333333333334
Recall: 0.625

Fold 2:
Accuracy: 0.8333333333333334
F1: 0.8
Precision: 0.8571428571428571
Recall: 0.75

Fold 3:
Accuracy: 0.7777777777777778
F1: 0.6666666666666666
Precision: 1.0
Recall: 0.5

Fold 4:
Accuracy: 1.0
F1: 1.0
Precision: 1.0
Recall: 1.0



In [21]:
from sklearn.model_selection import cross_validate

cv = cross_validate(pipeline, X, y, n_jobs=-1,
                    scoring=["accuracy",
                             "f1",
                             "precision",
                             "recall"])

In [22]:
for i in range(len(cv)-1):
    print(
        " accuracy", cv["test_accuracy"][i], "\n",
        "f1", cv["test_f1"][i], "\n",
        "recall", cv["test_recall"][i], "\n",
        "precision", cv["test_precision"][i], "\n"
    )
    print("---------------------------------------------")

 accuracy 0.9523809523809523 
 f1 0.9473684210526315 
 recall 0.9 
 precision 1.0 

---------------------------------------------
 accuracy 0.9047619047619048 
 f1 0.8888888888888888 
 recall 0.8 
 precision 1.0 

---------------------------------------------
 accuracy 0.85 
 f1 0.8 
 recall 0.6666666666666666 
 precision 1.0 

---------------------------------------------
 accuracy 0.85 
 f1 0.8235294117647058 
 recall 0.7777777777777778 
 precision 0.875 

---------------------------------------------
 accuracy 0.8 
 f1 0.75 
 recall 0.6666666666666666 
 precision 0.8571428571428571 

---------------------------------------------


In [23]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = pipeline.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

print(classification_report(y_test, y_pred))

Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         4
           1       1.00      1.00      1.00         7

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11



In [24]:
model = pipeline["svc"]

feature_importances = model.coef_.T.reshape(55,)
feature_names = X.columns

importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

importance_df

AttributeError: coef_ is only available when using a linear kernel